## Trabajaremos con la siguiente base de datos, donde debemos realizar un analisis sobre los comentarios sobre una aplicación, donde debemos poder predecir los comentarios negativos y positivo

importamos librerias necesarias

In [155]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from pickle import dump

### Lectura del archivo y verificacion si hay valores faltantes o duplicados


In [156]:
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv")
data = data.iloc[:, 1:]

In [157]:
data.isna().sum()

review      0
polarity    0
dtype: int64

In [158]:
data.duplicated().sum()

0

In [159]:
data.head()

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0


### Separamos los datos y y los vectorizamos.

In [160]:
X_train, X_test, y_train, y_test = train_test_split(data["review"], data["polarity"], test_size=0.2, random_state=42)

In [161]:
vectorizer = CountVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

In [162]:
sorter = MultinomialNB().fit(X_train_vec,y_train)         

y_pred = sorter.predict(X_test_vec)

In [163]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179



In [164]:
gaus = GaussianNB().fit(X_train_vec,y_train)

y_pred = gaus.predict(X_test_vec)

In [165]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179



In [166]:
bearn = BernoulliNB().fit(X_train_vec,y_train)

y_pred = bearn.predict(X_test_vec)

In [167]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.93      0.85       126
           1       0.70      0.40      0.51        53

    accuracy                           0.77       179
   macro avg       0.74      0.66      0.68       179
weighted avg       0.76      0.77      0.75       179



In [174]:
param_grid = {
    "alpha": np.linspace(0.01, 10.0, 100),
    "fit_prior": [True, False]
}

grid_search = RandomizedSearchCV(sorter, param_grid, cv=5, scoring='recall', n_jobs=-1, n_iter=400, random_state=42)

grid_search.fit(X_train_vec, y_train)

best_params = grid_search.best_params_

print("Mejores hiperparámetros:", best_params)

y_pred = grid_search.predict(X_test_vec)

print(classification_report(y_test, y_pred))


/home/vscode/.local/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 200 is smaller than n_iter=400. Running 200 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores hiperparámetros: {'fit_prior': False, 'alpha': 0.31272727272727274}
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       126
           1       0.72      0.68      0.70        53

    accuracy                           0.83       179
   macro avg       0.79      0.78      0.79       179
weighted avg       0.82      0.83      0.83       179



In [175]:
direccion = "/workspaces/Naive_bayes/models/mi_modelo.sav"
dump(sorter, open(direccion, 'wb'))


# Conclusión : 

    Pudimos determinar que el mejor modelo es el modelo Multinomial, el cual nos arrojo la siguiente información.

    º Para la Clase 0 (Comentarios Negativos):

        - Precisión (Precision): 87% de los comentarios clasificados como negativos fueron realmente negativos.

        - Exhaustividad (Recall): De todos los comentarios negativos reales, el modelo identificó correctamente el 89% de ellos.

        - Puntuación F1: La puntuación F1 es una combinación de precisión y recall, proporcionando un equilibrio. En este caso, es 88%.




Para la Clase 1 (Comentarios Positivos):

        - Precisión (Precision): 72% de los comentarios clasificados como positivos fueron realmente positivos.

        - Exhaustividad (Recall): De todos los comentarios positivos reales, el modelo identificó correctamente el 68% de ellos.
        
        - Puntuación F1: La puntuación F1 para comentarios positivos es 70%.




Interpretación General:

La precisión es alta para los comentarios negativos, lo que significa que cuando el modelo dice que un comentario es negativo, tiende a ser correcto en un 87% de las veces.
La exhaustividad es un poco más alta para los comentarios negativos (89%), lo que indica que el modelo identifica la mayoría de los comentarios negativos reales.
Para los comentarios positivos, tanto la precisión como la exhaustividad son un poco más bajas (72% y 68%, respectivamente), lo que sugiere que hay más margen de mejora en la identificación de comentarios positivos.